# ファイル操作

In [2]:
extern crate dirs;
use std::fs;
use std::env;
use std::path::Path;
use std::io::prelude::*;

#[cfg(unix)]
use std::os::unix::fs as fs_unix;
#[cfg(unix)]
use fs_unix::PermissionsExt;

#[cfg(windows)]
use std::os::windows::fs as fs_windows;

* カレントディレクトリをホームに変更

In [3]:
let home=dirs::home_dir();
if let Some(p) = home { env::set_current_dir(&p); }

()

標準ライブラリにも `env::home_dir()` があるが,動作が不安定なので非推奨であり, `dirs` クレートの `dirs::home_dir()` の使用が勧められている。

## ファイル/フォルダ/シンボリックリンクの作成

* Emptyという空フォルダを作成

In [4]:
fs::create_dir("Empty");

* 一時ディレクトリの場合
```Python
import tempfile
t=tempfile.TemporaryDirectory()
t.name # ディレクトリのパス
t.cleanup() # ディレクトリを削除する
```

* Blankという空ファイルを作成

In [5]:
fs::File::create("Blank");

* フォルダEmptyの中にファイルBlankのシンボリックリンクSymlinkを作成

In [6]:
#[cfg(unix)]
fs_unix::symlink("../Blank","Empty/Symlink");
#[cfg(windows)]
fs_windows::symlink_file("../Blank","Empty/Symlink");

- シンボリックリンクを作成する関数はプラットフォームによって異なるので,上記のようにマクロでプラットフォームに分けてインポートした。
	Windows の `symlink_file` はファイルへのリンクの場合のみに使用する。ディレクトリへのリンクは `std::os::windows::fs::symlink_dir` を使用する。
- ハードリンクの場合
```Rust
fs::hard_link("../Blank","Empty/Link")
```

## 書込み
* Untitled.mdというMarkdownファイルを作成して書込み

In [7]:
// 既にUntitled.mdが存在すれば上書きされる
let mut io=fs::OpenOptions::new()
	.write(true)
	.truncate(true)
	.create(true).open("Untitled.md")?;
io.write_all(b"# Header 1")?;

## 移動/名称変更

* フォルダEmptyをPackageに名称変更

In [8]:
fs::rename("Empty","Package")

Ok(())

* Packageフォルダ内のSymlinkファイルをAliasに名称変更

In [9]:
fs::rename("Package/Symlink","Package/Alias")

Ok(())

* Untitled.mdを移動して,名称変更

In [10]:
fs::rename("Untitled.md","Package/Headers.md")

Ok(())

## 追記
* Markdownファイルに追記

In [11]:
let mut io=fs::OpenOptions::new()
	.write(true)
	.append(true).open("Package/Headers.md")?;
io.write_all(b"\n## Header 2\n### Header 3");

## 読込み
* Markdownファイルを読込み

In [12]:
let mut io=fs::OpenOptions::new()
	.read(true).open("Package/Headers.md")?;
let mut data=String::new();
io.read_to_string(&mut data)?;
data;

## 再帰的にフォルダ作成
* フォルダを一気に作成

In [13]:
fs::create_dir_all("Package/Module/Submodule/Item");

## ディレクトリの内容を表示

In [14]:
let it=fs::read_dir("Package")?; // Iterator of DirEntry
// Converting DirEntry -> file name
let list:Vec<String> = it.map(|r| -> String {
    if let Ok(e)=r {
        if let Ok(s)=e.file_name().into_string() { return s; }
    }
    "?".to_string() // called when unable to get file name
}).collect();
list

["Alias", "Module", "Headers.md"]

- ワイルドカードを使って,条件を満たすファイルを見つけ出す場合は, `glob` クレートを使用する

## ファイルの複製
* Blankを複製

In [15]:
fs::copy("Blank","Package/Blank")?

0

## ファイル/フォルダの削除

* ファイルBlankを削除

In [16]:
fs::remove_file("Blank")?

()

* シンボリックリンクAliasを削除

In [17]:
fs::remove_file("Package/Alias")?

()

* フォルダModuleを削除

In [18]:
fs::remove_dir_all("Package/Module")?

()

- 空のフォルダは `fs::remmove_dir` で削除できる。空でない場合は削除できない。  
	`fs::remove_dir_all` を使えば,空でなくても削除できる

## 権限を確認/変更

* 状態を確認する関数

In [19]:
fn check(path:String) {
	let p=Path::new(&path);
	if p.exists() { println!("存在しています"); }
	else { println!("存在していません"); return; }

	let r=p.symlink_metadata(); // fs::symlink_metadata
    if r.is_err() { return; }
    let m=r.unwrap();
	let ft=m.file_type();
	if ft.is_symlink() {
		println!("シンボリックリンクです");
		let r=p.read_link();
		if let Ok(pb)=r {
			if let Some(s)=pb.to_str() {
 				println!("リンク先: {}",s);
			}
		}
		if p.is_file() { println!("リンク先はファイルです"); }
		if p.is_dir() { println!("リンク先はフォルダです"); }      
	}
	else {
		if ft.is_file() { println!("ファイルです"); }
		if ft.is_dir() { println!("フォルダです"); }
	}

	let pm=m.permissions();
	#[cfg(unix)]
	{
		let mode=pm.mode();
		if mode&0o400 != 0 { println!("読込可能です"); }
		if mode&0o200 != 0 { println!("書込可能です"); }
		if mode&0o100 != 0 { println!("実行可能です"); }
	}
	#[cfg(windows)]
	{
		if !pm.readonly() { println!("読込可能です"); }        
	}

	if m.len()==0 { println!("空です"); }
}

* Markdownファイルの現在の状態を確認

In [20]:
check("Package/Headers.md".to_string())

存在しています
ファイルです
読込可能です
書込可能です


()

* 読込/書込権限を剥奪

In [21]:
let mut pm=fs::metadata("Package/Headers.md")?.permissions();
#[cfg(unix)]
pm.set_mode(0o044);
#[cfg(windows)]
pm.set_readonly(false);

* 状態を確認

In [22]:
check("Package/Headers.md".to_string())

存在しています
ファイルです
読込可能です
書込可能です


()

* 読込/実行権限を付加

In [23]:
let mut pm=fs::metadata("Package/Headers.md")?.permissions();
#[cfg(unix)]
pm.set_mode(0o544);
#[cfg(windows)]
pm.set_readonly(true);

* 状態を確認

In [24]:
check("Package/Headers.md".to_string())

存在しています
ファイルです
読込可能です
書込可能です


()

- `Permissions.set_mode` は数値により権限を指定する
	* 4: 読込可能 (Readable)
	* 2: 書込可能 (Writable)
	* 1: 実行可能 (eXecutable)

- これらの和を3つ並べて指定する。  
    1つ目はユーザの権限,2つ目はゲストの権限,3つ目はその他の者の権限である。  
	Rustでは8進数で表記するために,前に `0o` を付加する

	e.g. `Permissions.set_mode(0o754)`
	* ユーザは 7 = 4+2+1 だから,読込可能,書込可能,実行可能
	* ゲストは 5 = 4 + 1 だから,読込可能,実行可能
	* その他は 4 だから,読込のみ可能